In [1]:
from qiskit.transpiler import InstructionDurations
from qiskit import IBMQ, transpile, Aer
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-france', group='univ-montpellier', project='default')
backend = provider.get_backend('ibmq_jakarta')

In [2]:
import os
os.getcwd()
os.chdir('/home/siyuan/Seafile/Thesis/PycharmProjects/DD_PE_paper/')

In [3]:
from src.tools.DD_insertion import construct_bv_circuit, \
                                 construct_graph_matrix, \
                                 convert_count_to_prob, \
                                 translate_circuit_to_basis
from qiskit.circuit.library import QFT, GraphState

In [4]:
bv_circuits = []
hs_circuits = []
qft_circuits = []
for i in range(3, 7):
    bv_circuits.append(construct_bv_circuit(i))

# for i in range(2, 15, 2):
#     hs_circuits.append(construct_hs_circuit(i))

for i in range(3, 7):
    qft_circuits.append(QFT(i))

for circuit in bv_circuits:
    circuit.measure_all()

# for circuit in hs_circuits:
#     circuit.measure_all()

for circuit in qft_circuits:
    circuit.measure_all()

In [5]:
durations = InstructionDurations.from_backend(backend)
## add duration of y gates which are used for DD sequences
bconf = backend.configuration()
for i in range(bconf.num_qubits):
    x_duration = durations.get('x', i)
    durations.update(InstructionDurations(
        [('y', i, x_duration)]
        ))

    durations.update(InstructionDurations(
        [('rx', i, x_duration)]
        ))

    durations.update(InstructionDurations(
        [('ry', i, x_duration)]
        ))

In [6]:
graph_state_circuits = []
coupling_map = backend.configuration().coupling_map

for i in range(3, 7):
    gs_circuit_matrix = construct_graph_matrix(i, coupling_map)
    graph_state_circuits.append(GraphState(gs_circuit_matrix))

for circuit in graph_state_circuits:
    circuit.measure_all()

In [7]:
from src.tools.DD_insertion import pm_DD_sequences
pms = pm_DD_sequences(durations)

In [8]:
bv_job_ids = []
bv_jobs = []

for circuit in bv_circuits:
    circuit_list = []
    transpiled_qc = transpile(circuit, backend=backend, optimization_level=3, seed_transpiler=1)
    circuit_list.append(transpiled_qc)
    for pm in pms:
        qc_transpile = pm.run(transpiled_qc)
        qc_transpile_base = translate_circuit_to_basis(qc_transpile, bconf)
        circuit_list.append(qc_transpile_base)
    job = backend.run(circuit_list, shots=8192)
    bv_jobs.append(job)
    job_id = job.job_id()
    print(job_id)
    bv_job_ids.append(job_id)

if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: False
6295d644bc0d026727dbecf8
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: False
6295d6469baeab12014acd73
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
6295d648bc0d029cefdbecf9
if applied DD for None: False
if applied DD f

In [9]:
for job_id in bv_job_ids:
    print(job_id)

6295d644bc0d026727dbecf8
6295d6469baeab12014acd73
6295d648bc0d029cefdbecf9
6295d64a0212ff553d251f18


In [7]:
from qiskit import execute

shots = 8192
psts = []
bv_job_ids = [
    '6295d644bc0d026727dbecf8',
    '6295d6469baeab12014acd73',
    '6295d648bc0d029cefdbecf9',
    '6295d64a0212ff553d251f18',


]
for i, circuit in enumerate(bv_circuits):
    ideal_counts = execute(circuit, backend=Aer.get_backend('qasm_simulator'), shots=shots).result().get_counts()
    ideal_result = list(ideal_counts.keys())[0]
    result_counts = backend.retrieve_job(bv_job_ids[i]).result().get_counts()
    pst = []
    for result_count in result_counts:
        obtain_results = result_count.get(ideal_result, 0)
        pst.append(obtain_results/shots)
    psts.append(pst)

bv_circuit_names = ['BV3', 'BV4', 'BV5', 'BV6']
DD_sequences = ['no_dd', 'hahn_X', 'hahn_Y', 'CP', 'CPMG', 'xy4', 'xy8', 'xy16', 'udd1', 'udd2', 'kdd']
bv_results_data = {}

for i, name in enumerate(bv_circuit_names):
    print(f'---------Result of circuit {name} -------------')
    for j, pst in enumerate(psts[i]):
        print(f'pst result of {DD_sequences[j]}: {pst}')
    bv_results_data[name] = psts[i]

---------Result of circuit BV3 -------------
pst result of no_dd: 0.451904296875
pst result of hahn_X: 0.458251953125
pst result of hahn_Y: 0.4482421875
pst result of CP: 0.470947265625
pst result of CPMG: 0.455322265625
pst result of xy4: 0.4619140625
pst result of xy8: 0.4654541015625
pst result of xy16: 0.45458984375
pst result of udd1: 0.446044921875
pst result of udd2: 0.453125
pst result of kdd: 0.4544677734375
---------Result of circuit BV4 -------------
pst result of no_dd: 0.4017333984375
pst result of hahn_X: 0.3922119140625
pst result of hahn_Y: 0.3916015625
pst result of CP: 0.39111328125
pst result of CPMG: 0.4063720703125
pst result of xy4: 0.40673828125
pst result of xy8: 0.4034423828125
pst result of xy16: 0.3935546875
pst result of udd1: 0.41064453125
pst result of udd2: 0.3958740234375
pst result of kdd: 0.3997802734375
---------Result of circuit BV5 -------------
pst result of no_dd: 0.37548828125
pst result of hahn_X: 0.3612060546875
pst result of hahn_Y: 0.35742187

In [8]:
bv_results_data

{'BV3': [0.451904296875,
  0.458251953125,
  0.4482421875,
  0.470947265625,
  0.455322265625,
  0.4619140625,
  0.4654541015625,
  0.45458984375,
  0.446044921875,
  0.453125,
  0.4544677734375],
 'BV4': [0.4017333984375,
  0.3922119140625,
  0.3916015625,
  0.39111328125,
  0.4063720703125,
  0.40673828125,
  0.4034423828125,
  0.3935546875,
  0.41064453125,
  0.3958740234375,
  0.3997802734375],
 'BV5': [0.37548828125,
  0.3612060546875,
  0.357421875,
  0.3721923828125,
  0.375244140625,
  0.3662109375,
  0.3638916015625,
  0.3616943359375,
  0.3809814453125,
  0.3819580078125,
  0.3599853515625],
 'BV6': [0.2523193359375,
  0.2559814453125,
  0.2537841796875,
  0.3223876953125,
  0.3206787109375,
  0.333251953125,
  0.3106689453125,
  0.307861328125,
  0.333251953125,
  0.3162841796875,
  0.2828369140625]}

In [9]:
# import numpy as np
# import matplotlib.pyplot as plt

# baseline = []

# data = []
# for name in bv_circuit_names:
#     result_data = bv_results_data[name]
#     baseline.append(result_data[0])
#     ratio = [i/result_data[0] for i in result_data[1:]]
#     data.append(ratio)


# data = np.array(data)

# # hahn_X = data[:, 0]
# # hahn_Y = data[:, 1]

# cp = data[:, 2]
# cpmg = data[:, 3]
# xy4 = data[:, 4]
# xy8 = data[:, 5]
# xy16 = list(data[2:, 6])
# xy16.insert(0, None)
# xy16.insert(0, None)
# udd_X = data[:, 7]
# udd_Y = data[:, 8]
# kdd = list(data[2:, 9])
# kdd.insert(0, None)
# kdd.insert(0, None)


# x_ticks = bv_circuit_names[:]
# X = np.arange(len(x_ticks))

# from matplotlib.pyplot import figure
# fig = figure(num=None, figsize=(8, 6), dpi=600, facecolor='w', edgecolor='k')
# ax = fig.add_axes([0,0,1,1])

# # plt.plot(X, hahn_Y, linestyle='--', marker='o', color='g', label='hahn_X')
# # plt.plot(X, hahn_X, linestyle='--', marker='s', color='m', label='hahn_Y')
# plt.plot(X, cp, linestyle='--', marker='o', color='orange', label='cp')
# plt.plot(X, cpmg, linestyle='--', marker='.', color='b', label='cpmg')
# plt.plot(X, xy4, linestyle='--', marker='^', color='r', label='xy4')
# plt.plot(X, xy8, linestyle='--', marker='X', color='g', label='xy8')
# plt.plot(X, xy16, linestyle='--', marker='h', color='c', label='xy16')
# plt.plot(X, udd_X, linestyle='--', marker='d', color='y', label='udd_x')
# plt.plot(X, udd_Y, linestyle='--', marker='+', color='k', label='udd_y')
# plt.plot(X, kdd, linestyle='--', marker='*', color='m', label='kdd')
# plt.axhline(1, color='tab:brown', lw=2)

# plt.legend(loc='best', fontsize=16)
# # ax.set_title('Relative PST result for BV circuit on ibmq_jakarta', fontsize=18)
# ax.set_xticks(X)
# ax.set_xticklabels(x_ticks, fontsize=14)
# ax.set_ylabel('Relative PST', fontsize=16)
# plt.savefig('TQE_Relat_PST_BV_jakarta_T_0530.pdf', bbox_inches='tight', pad_inches=0)

In [10]:
gs_job_ids = []
gs_jobs = []

for circuit in graph_state_circuits:
    circuit_list = []
    transpiled_qc = transpile(circuit, backend=backend, optimization_level=3, seed_transpiler=1)
    circuit_list.append(transpiled_qc)
    for pm in pms:
        qc_transpile = pm.run(transpiled_qc)
        qc_transpile_base = translate_circuit_to_basis(qc_transpile, bconf)
        circuit_list.append(qc_transpile_base)
    job = backend.run(circuit_list, shots=8192)
    gs_jobs.append(job)
    job_id = job.job_id()
    print(job_id)
    gs_job_ids.append(job_id)

if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
6295d64b3bb4af8a4cb95509
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
6295d64d4bb9758357efb4ce
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
6295d64e4ad65509779e4aa6
if applied DD for None: False
if applied DD for N

In [11]:
for job_id in gs_job_ids:
    print(job_id)

6295d64b3bb4af8a4cb95509
6295d64d4bb9758357efb4ce
6295d64e4ad65509779e4aa6
6295d650f33cdba1eb2e069f


In [10]:
from scipy.spatial.distance import jensenshannon
from qiskit import execute
gs_jsds = []

gs_job_ids = [
    '6295d64b3bb4af8a4cb95509',
    '6295d64d4bb9758357efb4ce',
    '6295d64e4ad65509779e4aa6',
    '6295d650f33cdba1eb2e069f',
]


shots = 8192

for i, circuit in enumerate(graph_state_circuits):
    ideal_counts = execute(circuit, backend=Aer.get_backend('qasm_simulator'), shots=shots).result().get_counts()
    result_counts = backend.retrieve_job(gs_job_ids[i]).result().get_counts()
    jsd = []
    for result_count in result_counts:
        p_ideal, p_result = convert_count_to_prob(result_count, ideal_counts, shots=shots)
        jsd.append(jensenshannon(p_ideal, p_result))
    gs_jsds.append(jsd)

circuit_names = ['GS3', 'GS4', 'GS5', 'GS6',]
DD_sequences = ['no_dd', 'hahn_X', 'hahn_Y', 'CP', 'CPMG', 'xy4', 'xy8', 'xy16', 'udd1', 'udd2', 'kdd']

gs_results_data = {}

for i, name in enumerate(circuit_names):
    print(f'---------Result of circuit {name} -------------')
    for j, jsd in enumerate(gs_jsds[i]):
        print(f'jsd result of {DD_sequences[j]}: {jsd}')
    gs_results_data[name] = gs_jsds[i]

---------Result of circuit GS3 -------------
jsd result of no_dd: 0.045617911157655476
jsd result of hahn_X: 0.04873148817580413
jsd result of hahn_Y: 0.043520196766198276
jsd result of CP: 0.045116675284664974
jsd result of CPMG: 0.04899772517386398
jsd result of xy4: 0.04352061889499965
jsd result of xy8: 0.046582831962408686
jsd result of xy16: 0.04934130702368998
jsd result of udd1: 0.0524748453035879
jsd result of udd2: 0.0459955792053988
jsd result of kdd: 0.04865664748416749
---------Result of circuit GS4 -------------
jsd result of no_dd: 0.04532541315033542
jsd result of hahn_X: 0.041833688467098534
jsd result of hahn_Y: 0.04416170733155313
jsd result of CP: 0.04149466935349644
jsd result of CPMG: 0.0344696624420985
jsd result of xy4: 0.041766846368211374
jsd result of xy8: 0.04813045909338863
jsd result of xy16: 0.03652028017132206
jsd result of udd1: 0.042222097316990344
jsd result of udd2: 0.04524597834360879
jsd result of kdd: 0.0431737528886942
---------Result of circuit 

In [11]:
gs_results_data

{'GS3': [0.045617911157655476,
  0.04873148817580413,
  0.043520196766198276,
  0.045116675284664974,
  0.04899772517386398,
  0.04352061889499965,
  0.046582831962408686,
  0.04934130702368998,
  0.0524748453035879,
  0.0459955792053988,
  0.04865664748416749],
 'GS4': [0.04532541315033542,
  0.041833688467098534,
  0.04416170733155313,
  0.04149466935349644,
  0.0344696624420985,
  0.041766846368211374,
  0.04813045909338863,
  0.03652028017132206,
  0.042222097316990344,
  0.04524597834360879,
  0.0431737528886942],
 'GS5': [0.04738551089039782,
  0.051690452062911896,
  0.04775952025774626,
  0.041186840331897324,
  0.04329372304154203,
  0.04304837149649725,
  0.04926140996516049,
  0.0360560552050773,
  0.03822431653342749,
  0.04269122025950977,
  0.04392174199114971],
 'GS6': [0.06448677084140488,
  0.06844422494521789,
  0.07035494563456145,
  0.06667347348768611,
  0.07053164409642122,
  0.07053783518168275,
  0.07043332053304391,
  0.07043705315057246,
  0.0648477029668213,


In [12]:
# baseline = []

# data = []
# for name in circuit_names:
#     result_data = gs_results_data[name]
#     baseline.append(result_data[0])
#     ratio = [result_data[0]/i for i in result_data[1:]]
#     data.append(ratio)


# data = np.array(data)
# #
# # hahn_X = data[:, 0]
# # hahn_Y = data[:, 1]
# cp = data[:, 2]
# cpmg = data[:, 3]
# xy4 = data[:, 4]
# xy8 = data[:, 5]
# xy16 = data[:, 6]
# udd_X = data[:, 7]
# udd_Y = data[:, 8]
# kdd = data[:, 9]

# x_ticks = circuit_names
# X = np.arange(len(x_ticks))

# from matplotlib.pyplot import figure
# fig = figure(num=None, figsize=(8, 6), dpi=600, facecolor='w', edgecolor='k')
# ax = fig.add_axes([0,0,1,1])

# # plt.plot(X, hahn_Y, linestyle='--', marker='o', color='g', label='hahn_X')
# # plt.plot(X, hahn_X, linestyle='--', marker='s', color='m', label='hahn_Y')
# plt.plot(X, cp, linestyle='--', marker='o', color='orange', label='cp')
# plt.plot(X, cpmg, linestyle='--', marker='.', color='b', label='cpmg')
# plt.plot(X, xy4, linestyle='--', marker='^', color='r', label='xy4')
# plt.plot(X, xy8, linestyle='--', marker='X', color='g', label='xy8')
# plt.plot(X, xy16, linestyle='--', marker='h', color='c', label='xy16')
# plt.plot(X, udd_X, linestyle='--', marker='d', color='y', label='udd_x')
# plt.plot(X, udd_Y, linestyle='--', marker='+', color='k', label='udd_y')
# plt.plot(X, kdd, linestyle='--', marker='*', color='m', label='kdd')
# plt.axhline(1, color='tab:brown', lw=2)

# plt.legend(loc='best', fontsize=16)
# # ax.set_title('Relative JSD result for GS circuit on ibmq_jakarta', fontsize=18)
# ax.set_xticks(X)
# ax.set_xticklabels(x_ticks, fontsize=14)
# ax.set_ylabel('Relative JSD', fontsize=16)

# plt.savefig('TQE_Relat_JSD_GS_jakarta_0530.pdf', bbox_inches='tight', pad_inches=0)

In [12]:
qft_job_ids = []
qft_jobs = []

for circuit in qft_circuits:
    circuit_list = []
    transpiled_qc = transpile(circuit, backend=backend, optimization_level=3, seed_transpiler=1)
    circuit_list.append(transpiled_qc)
    for pm in pms:
        qc_transpile = pm.run(transpiled_qc)
        qc_transpile_base = translate_circuit_to_basis(qc_transpile, bconf)
        circuit_list.append(qc_transpile_base)
    job = backend.run(circuit_list, shots=8192)
    qft_jobs.append(job)
    job_id = job.job_id()
    print(job_id)
    qft_job_ids.append(job_id)

if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
6295d658f33cdb01d82e06a0
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
6295d65a9baeab34e64acd75
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
6295d65c0212ffa0cd251f1a
if applied DD for None: False
if applied DD for N

In [13]:
for job_id in qft_job_ids:
    print(job_id)

6295d658f33cdb01d82e06a0
6295d65a9baeab34e64acd75
6295d65c0212ffa0cd251f1a
6295d66016cdc41a90f25e45


In [13]:
from scipy.spatial.distance import jensenshannon
qft_jsds = []

qft_job_ids = [
    '6295d658f33cdb01d82e06a0',
    '6295d65a9baeab34e64acd75',
    '6295d65c0212ffa0cd251f1a',
    '6295d66016cdc41a90f25e45',
]

shots = 8192

for i, circuit in enumerate(qft_circuits):
    ideal_counts = execute(circuit, backend=Aer.get_backend('qasm_simulator'), shots=shots).result().get_counts()
    result_counts = backend.retrieve_job(qft_job_ids[i]).result().get_counts()
    jsd = []
    for result_count in result_counts:
        p_ideal, p_result = convert_count_to_prob(result_count, ideal_counts, shots=shots)
        jsd.append(jensenshannon(p_ideal, p_result))
    qft_jsds.append(jsd)

circuit_names = ['QFT3', 'QFT4', 'QFT5', 'QFT6', ]
DD_sequences = ['no_dd', 'hahn_X', 'hahn_Y', 'CP', 'CPMG', 'xy4', 'xy8', 'xy16', 'udd1', 'udd2', 'kdd']

qft_results_data = {}

for i, name in enumerate(circuit_names):
    print(f'---------Result of circuit {name} -------------')
    for j, jsd in enumerate(qft_jsds[i]):
        print(f'jsd result of {DD_sequences[j]}: {jsd}')
    qft_results_data[name] = qft_jsds[i]

---------Result of circuit QFT3 -------------
jsd result of no_dd: 0.057820524346515294
jsd result of hahn_X: 0.054389690381959406
jsd result of hahn_Y: 0.0480284484922724
jsd result of CP: 0.0553627414815798
jsd result of CPMG: 0.05964022163874728
jsd result of xy4: 0.05465773454819876
jsd result of xy8: 0.07642216764586765
jsd result of xy16: 0.04817951243898198
jsd result of udd1: 0.0491762951306439
jsd result of udd2: 0.07941200376483583
jsd result of kdd: 0.06063438548432425
---------Result of circuit QFT4 -------------
jsd result of no_dd: 0.12262137139010473
jsd result of hahn_X: 0.12026678356298967
jsd result of hahn_Y: 0.1268416947422105
jsd result of CP: 0.12284752318409749
jsd result of CPMG: 0.09637120001886744
jsd result of xy4: 0.10091114060481657
jsd result of xy8: 0.16294372452124287
jsd result of xy16: 0.11244325163912486
jsd result of udd1: 0.11565427838239738
jsd result of udd2: 0.11860921139947361
jsd result of kdd: 0.11830818085373862
---------Result of circuit QFT

In [14]:
qft_results_data

{'QFT3': [0.057820524346515294,
  0.054389690381959406,
  0.0480284484922724,
  0.0553627414815798,
  0.05964022163874728,
  0.05465773454819876,
  0.07642216764586765,
  0.04817951243898198,
  0.0491762951306439,
  0.07941200376483583,
  0.06063438548432425],
 'QFT4': [0.12262137139010473,
  0.12026678356298967,
  0.1268416947422105,
  0.12284752318409749,
  0.09637120001886744,
  0.10091114060481657,
  0.16294372452124287,
  0.11244325163912486,
  0.11565427838239738,
  0.11860921139947361,
  0.11830818085373862],
 'QFT5': [0.0860940696998467,
  0.09080589331741822,
  0.09128982299214157,
  0.08621551942028059,
  0.11954902641954948,
  0.10703508981800557,
  0.09597140679727785,
  0.12509616955358904,
  0.0934574405754649,
  0.1211135157071287,
  0.11704182483355535],
 'QFT6': [0.10329411581918174,
  0.1025310242760335,
  0.10021505095527729,
  0.09828464020225441,
  0.09134927102134534,
  0.0869879227518163,
  0.09457413979194952,
  0.10664141667855047,
  0.10849962139658625,
  0.09

In [15]:
# import numpy as np
# import matplotlib.pyplot as plt


# baseline = []
# circuit_names = ['QFT3', 'QFT4', 'QFT5', 'QFT6']
# data = []
# for name in circuit_names:
#     result_data = qft_results_data[name]
#     baseline.append(result_data[0])
#     ratio = [result_data[0]/i for i in result_data[1:]]
#     data.append(ratio)


# data = np.array(data)
# #
# # hahn_X = data[:, 0]
# # hahn_Y = data[:, 1]
# cp = data[:, 2]
# cpmg = data[:, 3]
# xy4 = data[:, 4]
# xy8 = data[:, 5]
# xy16 = data[:, 6]
# udd_X = data[:, 7]
# udd_Y = data[:, 8]
# kdd = data[:, 9]

# x_ticks = circuit_names
# X = np.arange(len(x_ticks))

# from matplotlib.pyplot import figure
# fig = figure(num=None, figsize=(8, 6), dpi=600, facecolor='w', edgecolor='k')
# ax = fig.add_axes([0,0,1,1])

# # plt.plot(X, hahn_Y, linestyle='--', marker='o', color='g', label='hahn_X')
# # plt.plot(X, hahn_X, linestyle='--', marker='s', color='m', label='hahn_Y')
# plt.plot(X, cp, linestyle='--', marker='o', color='orange', label='cp')
# plt.plot(X, cpmg, linestyle='--', marker='.', color='b', label='cpmg')
# plt.plot(X, xy4, linestyle='--', marker='^', color='r', label='xy4')
# plt.plot(X, xy8, linestyle='--', marker='X', color='g', label='xy8')
# plt.plot(X, xy16, linestyle='--', marker='h', color='c', label='xy16')
# plt.plot(X, udd_X, linestyle='--', marker='d', color='y', label='udd_x')
# plt.plot(X, udd_Y, linestyle='--', marker='+', color='k', label='udd_y')
# plt.plot(X, kdd, linestyle='--', marker='*', color='m', label='kdd')
# plt.axhline(1, color='tab:brown', lw=2)

# plt.legend(loc='best', fontsize=16)
# # ax.set_title('Relative JSD result for QFT circuit on ibmq_jakarta', fontsize=18)
# ax.set_xticks(X)
# ax.set_xticklabels(x_ticks, fontsize=14)
# ax.set_ylabel('Relative JSD', fontsize=16)

# plt.savefig('TQE_Relat_JSD_QFT_jakarta_0530.pdf', bbox_inches='tight', pad_inches=0)